In [43]:
# task 4 
#!pip install cirq tensorflow_quantum tensorflow
#!pip install sympy
import cirq,sympy
import tensorflow_quantum as tfq
import tensorflow as tf
import numpy as np

In [10]:
import numpy as np

# load the NPZ file
data = np.load('QIS_EXAM_200Events.npz',allow_pickle=True)

# see the keys
print(data.keys())

KeysView(NpzFile 'QIS_EXAM_200Events.npz' with keys: training_input, test_input)


In [12]:
# we cannot clearly see the signal and background, so it should be emmbedded inside it

training_obj = data['training_input'].item()
test_obj = data['test_input'].item()  # extract the underlying object
print(type(training_obj))
print(type(test_obj))
print(training_obj.keys() if hasattr(training_obj, 'keys') else training_obj)

# now we can combine the data since its its two different objects
X_train = np.concatenate((training_obj['0'], training_obj['1']))
y_train = np.concatenate((np.zeros(len(training_obj['0'])), np.ones(len(training_obj['1']))))

# combine test data
X_test = np.concatenate((test_obj['0'], test_obj['1']))
y_test = np.concatenate((np.zeros(len(test_obj['0'])), np.ones(len(test_obj['1']))))

<class 'dict'>
<class 'dict'>
dict_keys(['0', '1'])


In [37]:
# now its time encode the data :)))
# since our data is already normalized between -1 and 1, we can map a value x in [-1,1] to an angle θ in [0,π]

def create_angle_embedding_circuit(qubits, features):
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        # scaling data value to angle
        angle = features[i] * np.pi
        # apply an RX gate with the calculated angle
        circuit.append(cirq.rx(angle)(qubit))
    return circuit

# one qubit per feature to encode the data
num_qubits = 5
qubits = [cirq.GridQubit(0, i) for i in range(num_qubits)]


sample_features = X_train[0]

# creating the encoding circuit
circuit = create_angle_embedding_circuit(qubits, sample_features)
print(circuit)

(0, 0): ───Rx(-0.431π)───

(0, 1): ───Rx(0.868π)────

(0, 2): ───Rx(-0.926π)───

(0, 3): ───Rx(-0.927π)───

(0, 4): ───Rx(-0.569π)───


In [45]:
# the classical data now encoded into quantum states, now we can do the fun part (QGAN)!!!!!!!
# first, lets start with discriminators


# for further rotations, we need to create a parametrized circuit
def create_parametrized_circuit(qubits, symbols):
    circuit = cirq.Circuit()
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.ry(symbols[i])(qubit))
    return circuit

# Example: define symbols for 5 qubits (adjust number as needed)
symbols = sympy.symbols('theta0:5')

# combining the encoding and parameterized circuit like its magic!!!!
encoding_circuit = create_angle_embedding_circuit(qubits, sample_features)
param_circuit = create_parametrized_circuit(qubits, symbols)
full_discriminator_circuit = encoding_circuit + param_circuit

# define an observable to measure; for instance, measure Z(it is not important though) on the first qubit ; 
observable = cirq.Z(qubits[0])

# creating TFQ layer for the discriminator (its for calculating expected value and optimizing trainable params)
discriminator_model = tf.keras.Sequential([
    tfq.layers.PQC(full_discriminator_circuit, observable),
    tf.keras.layers.Dense(1, activation='sigmoid')  # to output a probability
])

discriminator_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                            loss='binary_crossentropy',
                            metrics=['accuracy'])

In [86]:
# after wrapping the model with TFQ, its time for generator!
# we must ensure that the generator produces a noise value, then we will be cool to wrap it with TFQ too ;)

qubits = [cirq.GridQubit(0, i) for i in range(5)]

def create_generator_circuit(qubits, symbols):
    circuit = cirq.Circuit()
    
    for i, qubit in enumerate(qubits):
        circuit.append(cirq.ry(symbols[i])(qubit))
    # P.S : I did not see any reason to entangle the qubits.
    return circuit

# one qubit per feature as usual
gen_symbols = sympy.symbols('phi0:5')

generator_circuit = create_generator_circuit(qubits, gen_symbols)
print("Generator circuit:")
print(generator_circuit)


Generator circuit:
(0, 0): ───Ry(phi0)───

(0, 1): ───Ry(phi1)───

(0, 2): ───Ry(phi2)───

(0, 3): ───Ry(phi3)───

(0, 4): ───Ry(phi4)───


In [88]:
# wrapping generator with TFQ
generator_model = tf.keras.Sequential([
    tfq.layers.PQC(generator_circuit, cirq.Z(qubits[0])),
    tf.keras.layers.Dense(5, activation='tanh') 
])

generator_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                        loss='binary_crossentropy') 

In [90]:
# before the training cycle, we need to define some helper functions:

def get_real_data_batch(batch_size):
    indices = np.random.choice(len(X_train), batch_size, replace=False)
    return X_train[indices]

def generate_noise(batch_size, noise_dim):
    # noise_dim = size of the noise vector
    return np.random.normal(0, 1, (batch_size, noise_dim))

In [102]:
# ping-pong begins here
# some parameters
num_epochs = 1000
noise_dim = 5 # In classical GANs, usually noise_dim is around 100 but in this case i prefer to go with feature size
batch_size = 16 # I wouldn't use this if i had a Google Colab Pro
steps_per_epoch = int(len(X_train) / batch_size)

# training cycle
for epoch in range(num_epochs):
    d_loss_epoch = [] # discriminator loss
    g_loss_epoch = [] # generator loss
    
    for step in range(steps_per_epoch):
        # ---------------------
        #1. First we need to train the discriminator
        # ---------------------
        real_data = get_real_data_batch(batch_size)
        # creating fake date with generator using random noise
        noise = generate_noise(batch_size, noise_dim)  # random noise
        fake_data = generator_model.predict(noise)
        
        # combine real and fake data to label them, 1 for real, 0 for fake
        combined_data = np.concatenate([real_data, fake_data], axis=0)
        combined_labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))], axis=0)
        
        # train the discriminator
        d_loss = discriminator_model.train_on_batch(combined_data, combined_labels)
        d_loss_epoch.append(d_loss)
        
        # ---------------------------
        # 2. Now, generator training part
        # ---------------------------
        # since generator's goal is to fool the discriminator, it uses misleading labels (1)

        noise = generate_noise(batch_size,noise_dim)
        misleading_labels = np.ones((batch_size, 1))
        
        # Thanks to TFQ, discriminators weights are not updated during generator training but only generator weights are updated
        g_loss = combined_model.train_on_batch(noise, misleading_labels)
        g_loss_epoch.append(g_loss)
    
    # saving a report every epoch
    avg_d_loss = np.mean(d_loss_epoch)
    avg_g_loss = np.mean(g_loss_epoch)
    print(f"Epoch {epoch+1}/{num_epochs} - Discriminator Loss: {avg_d_loss:.4f}, Generator Loss: {avg_g_loss:.4f}")

TypeError: in user code:

    File "/root/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/root/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/root/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/root/venv/lib/python3.10/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/root/venv/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filemy2hksps.py", line 11, in tf__call
        circuit_batch_dim = ag__.converted_call(ag__.ld(tf).gather, (ag__.converted_call

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=15ae0520-e0c3-4a6b-83df-cfc388c1a187' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>